In [6]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# 대시보드 앱 초기화
app = dash.Dash(__name__)

# 예시 데이터 준비
toner_data = {
    'Brand': ['HP Toner', 'HP Toner', 'HP Toner', 'HP Toner',
              'Samsung Toner', 'Samsung Toner', 'Samsung Toner', 'Samsung Toner',
              'Canon Toner', 'Canon Toner', 'Canon Toner', 'Canon Toner',
              'Epson Toner', 'Epson Toner', 'Epson Toner', 'Epson Toner',
              'Brother Toner', 'Brother Toner', 'Brother Toner', 'Brother Toner'],
    'Color': ['Black', 'Red', 'Blue', 'Yellow'] * 5,
    'Stock': [50, 120, 70, 30,
              120, 60, 40, 140,
              180, 40, 60, 20,
              110, 55, 85, 35,
              90, 145, 65, 25]
}

df_stock = pd.DataFrame(toner_data)

# 브랜드별 색상 토너의 총 재고 계산
total_stock_by_color = df_stock.groupby('Color')['Stock'].sum().reset_index()

# CSS 스타일 정의 (개선된 테이블 스타일)
table_style = {
    'border-collapse': 'collapse',
    'width': '100%',
    'margin': '20px 0',
    'font-size': '16px',
    'text-align': 'center',
    'border': '1px solid #ddd',
    'box-shadow': '0 4px 8px rgba(0, 0, 0, 0.1)',
}

th_td_style = {
    'border': '1px solid #ddd',
    'padding': '12px',
    'background-color': '#f8f9fa',  # 더 밝은 배경색
    'color': '#333',  # 텍스트 색상
    'font-weight': 'bold',
    'text-transform': 'uppercase',  # 대문자 변환
    'letter-spacing': '0.05em',  # 글자 사이 간격
}

td_style = {
    'border': '1px solid #ddd',
    'padding': '12px',
    'background-color': '#ffffff',  # 흰색 배경
    'color': '#333',
    'font-weight': 'normal',
}

# 레이아웃 정의
app.layout = html.Div(children=[
    html.H1(children='Toner Stock Dashboard'),

    html.Div(children='''This dashboard shows the toner stock levels by brand and color.'''),

    dcc.Dropdown(
        id='brand-selector',
        options=[
            {'label': 'HP Toner', 'value': 'HP Toner'},
            {'label': 'Samsung Toner', 'value': 'Samsung Toner'},
            {'label': 'Canon Toner', 'value': 'Canon Toner'},
            {'label': 'Epson Toner', 'value': 'Epson'},
            {'label': 'Brother Toner', 'value': 'Brother'},
        ],
        value='HP Toner'
    ),

    dcc.Graph(
        id='brand-stock-graph'
    ),

    dcc.Graph(
        id='total-stock-graph'
    ),

    # 표 추가 (브랜드별 색상 토너 재고량)
    html.H2(children='Toner Stock Table by Brand and Color'),
    html.Table(id='stock-table', children=[
        html.Tr([html.Th('Brand', style=th_td_style), 
                 html.Th('Black', style=th_td_style), 
                 html.Th('Red', style=th_td_style), 
                 html.Th('Blue', style=th_td_style), 
                 html.Th('Yellow', style=th_td_style)]),
        *[
            html.Tr([html.Td(brand, style=td_style)] + 
                    [html.Td(df_stock[(df_stock['Brand'] == brand) & (df_stock['Color'] == color)]['Stock'].values[0], style=td_style) 
                     for color in ['Black', 'Red', 'Blue', 'Yellow']])
            for brand in df_stock['Brand'].unique()
        ]
    ], style=table_style)
])

# 콜백 설정 - 선택된 브랜드의 색상별 재고 그래프 업데이트
@app.callback(
    Output('brand-stock-graph', 'figure'),
    [Input('brand-selector', 'value')]
)
def update_brand_stock_graph(selected_brand):
    filtered_df = df_stock[df_stock['Brand'] == selected_brand]
    fig = px.bar(filtered_df, x='Color', y='Stock', title=f'{selected_brand} Toner Stock by Color', color='Color',
                 color_discrete_map={'Black': 'rgba(0, 0, 0, 0.6)', 'Red': 'rgba(255, 0, 0, 0.6)', 'Blue': 'rgba(0, 0, 255, 0.6)', 'Yellow': 'rgba(255, 255, 0, 0.6)'})

    fig.update_layout(
        xaxis_title='Color',
        yaxis_title='Stock Level',
        title_x=0.5,
        template='plotly_white'
    )
    return fig

# 콜백 설정 - 색상별 총 토너 재고 그래프 업데이트
@app.callback(
    Output('total-stock-graph', 'figure'),
    [Input('brand-selector', 'value')]
)
def update_total_stock_graph(selected_brand):
    fig = px.bar(total_stock_by_color, x='Color', y='Stock', title='Total Toner Stock by Color', color='Color',
                 category_orders={'Color': ['Black', 'Red', 'Blue', 'Yellow']},
                 color_discrete_map={'Black': 'rgba(0, 0, 0, 0.6)', 'Red': 'rgba(255, 0, 0, 0.6)', 'Blue': 'rgba(0, 0, 255, 0.6)', 'Yellow': 'rgba(255, 255, 0, 0.6)'})

    fig.update_layout(
        xaxis_title='Color',
        yaxis_title='Total Stock Level',
        title_x=0.5,
        template='plotly_white'
    )
    return fig

# 서버 실행
if __name__ == '__main__':
    app.run_server(debug=True)